In [1]:
import numba
import numpy as np


**Funciones de prueba:**

In [2]:
@numba.njit
def Sphere(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función de la esfera en el vector.
    Se usa en el intervalo de búsqueda [-5.12, 5.12] y su óptimo global está en f([0,..,0])=0

    Ejemplo de uso:
        > Sphere(np.array([0, 0]))

        > 0
    """
    suma = 0.0  # Usar un float para la suma
    for x in arr:
        suma += x ** 2
    return suma

intervaloSphere = (-5.12, 5.12)


# Ejemplo de uso (y compilación AOT)
resultado = Sphere(np.array([1.0, 2.0, 3.0]))
print(resultado)


14.0


In [3]:
@numba.njit
def Ackley(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función Ackley en el vector.
    Se usa en el intervalo de búsqueda [-30, 30] y su óptimo global está en f([0,..,0])=0.

    Ejemplo de uso:
        > Ackley(np.array([0, 0]))

        > 0
    """
    suma1 = 0.0
    suma2 = 0.0
    n = arr.shape[0]  # Usar el tamaño del array de NumPy
    for x in arr:
        suma1 += x ** 2
        suma2 += np.cos(2 * np.pi * x)
    
    # Calcular el resultado de la función Ackley
    term1 = 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt(suma1 / n))
    term2 = np.exp(suma2 / n)
    return term1 - term2

intervaloAckley = (-30, 30)

# Ejemplo de uso (y compilación AOT)
resultado = Ackley(np.array([0, 0]))
print(resultado)  # Debería imprimir un valor cercano a 0


-4.440892098500626e-16


In [4]:
@numba.njit
def Griewank(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función Griewank en el vector.
    Se usa en el intervalo de búsqueda [-600, 600] y su óptimo global está en f([0,..,0])=0.

    Ejemplo de uso:
        > Griewank(np.array([0, 0]))

        > 0
    """
    suma = 0.0
    m = 1.0  # Inicializar en 1 para la multiplicación de cosenos
    n = arr.shape[0]  # Obtener el número de elementos en el array

    for i in range(1, n + 1):
        x_i = arr[i - 1]  # Acceder al elemento en el índice correspondiente
        suma += (x_i ** 2) / 4000.0  # Sumar la parte de la función
        m *= np.cos(x_i / np.sqrt(i))  # Multiplicar por el coseno

    return 1 + suma - m

intervaloGriewank = (-600, 600)

# Ejemplo de uso (y compilación AOT)
resultado = Griewank(np.array([0, 0]))
print(resultado)  # Debería imprimir 0.0


0.0


In [5]:
@numba.njit
def Rastrigin(arr):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un np.array,
    y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
    Tiene mínimo global en f([0,..,0]).

    Ejemplo de uso:
        > Rastrigin(np.array([2, 3, 6, 7]))

        > 98.0
    """
    n = arr.shape[0]  # Obtener el número de elementos en el array
    suma = 0.0

    for x in arr:
        suma += x ** 2 - 10 * np.cos(2 * np.pi * x)  # Calcular la suma de la función Rastrigin

    return 10 * n + suma  # Retornar el resultado final

intervaloRastrigin = (-5.12, 5.12)

# Ejemplo de uso (y compilación AOT)
resultado = Rastrigin(np.array([2, 3, 6, 7]))
print(resultado)  # Debería imprimir 98.0


98.0


In [6]:
@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

# Ejemplo de uso (y compilación AOT)
resultado = Rosenbrock(np.array([1, 1]))
print(resultado)  # Debería imprimir 0.0


0.0


**Parámetros:**

In [7]:
numParticulas = 70
maxIteraciones = 1000
dimension = 10
w = 0.729
c1 = 1.49445
c2 = 1.49445 
funcionObjetivo = Rastrigin
intervalo = intervaloRastrigin


**Implementación PSO**

In [8]:
@numba.njit
def PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2):
    
    # Inicializamos nuestra población inicial de partículas y velocidades:
    a,b = intervalo
    particulas = np.random.uniform(a, b, (numParticulas, dimension))
    velocidades = np.zeros((numParticulas, dimension))

    # Inicializamos las mejores posiciones y mejores valores:
    mejores_posiciones = np.copy(particulas) 
    
    mejores_aptitudes  = np.empty(numParticulas)
    for i in range(numParticulas):
        mejores_aptitudes[i] = funcionObjetivo(particulas[i])

    mejor_posicion_enjambre = mejores_posiciones[np.argmin(mejores_aptitudes)]
    mejor_aptitud_enjambre = np.min(mejores_aptitudes)

    # Iteramos hasta maxIteraciones, actualizando la velocidad y la posición de cada partícula en cada iteración:
    for _ in range(maxIteraciones):
        # Actualizamos las velocidades:
        r1 = np.random.uniform(0, 1, (numParticulas, dimension))
        r2 = np.random.uniform(0, 1, (numParticulas, dimension))
        for i in range(numParticulas):
            for d in range(dimension):
                velocidades[i, d] = (w * velocidades[i, d] + c1 * r1[i, d] * (mejores_posiciones[i, d] - particulas[i, d]) + c2 * r2[i, d] * (mejor_posicion_enjambre[d] - particulas[i, d]))

        # Actualizamos las posiciones:
        for i in range(numParticulas):
            for d in range(dimension):
                particulas[i, d] += velocidades[i, d]

        # Evaluamos la aptitud de cada partícula:
        evaluaciones_de_aptitud = np.empty(numParticulas)
        for i in range(numParticulas):
            evaluaciones_de_aptitud[i] = funcionObjetivo(particulas[i])

        # Actualizamos las posiciones y las aptitudes:
        for i in range(numParticulas):
            if evaluaciones_de_aptitud[i] < mejores_aptitudes[i]:
                mejores_aptitudes[i] = evaluaciones_de_aptitud[i]
                mejores_posiciones[i] = particulas[i]

        # Actualizamos la mejor posición global si es que encontramos una mejor:
        mejor_aptitud_actual = np.min(evaluaciones_de_aptitud)
        if mejor_aptitud_actual < mejor_aptitud_enjambre:
            mejor_aptitud_enjambre = mejor_aptitud_actual
            mejor_posicion_enjambre = particulas[np.argmin(evaluaciones_de_aptitud)]
    
    # Retornamos la mejor solución encontrada por el PSO:
    return mejor_posicion_enjambre, mejor_aptitud_enjambre


In [9]:
PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([-2.78511345e-09, -4.86045870e-10,  4.66731648e-09,  1.22319179e-10,
         1.16191658e-09,  6.84645662e-09, -3.71762246e-10, -9.94958630e-01,
        -9.94958639e-01, -9.94958640e-01]),
 2.9848771712798907)

In [10]:
numParticulas = 70
maxIteraciones = 1000
dimension = 10
w = 0.729
c1 = 1.49445
c2 = 1.49445 
funcionObjetivo = Rosenbrock
intervalo = intervaloRosenbrock

PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([1.00043913, 1.00113574, 1.00227934, 1.00456311, 1.00950058,
        1.01913241, 1.03813089, 1.077247  , 1.16029399, 1.34602214]),
 0.03367485244627191)

In [11]:
numParticulas = 70 # Misma población que en AG para que la comparación sea justa.
maxIteraciones = 1000 # Mismas iteraciones que en AG para que la comparación sea justa.
dimension = 10 # Misma dimensión que en AG para que la comparación sea justa.
w = 0.729
c1 = 1.49445
c2 = 1.49445 
funcionObjetivo = Ackley
intervalo = intervaloAckley

PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([ 1.10990928e-12,  1.13417880e-12,  1.28659354e-12,  1.04817149e-11,
         2.05646379e-12,  1.92473812e-12,  2.22046771e-14, -3.22883344e-12,
        -1.96553070e-12, -3.23530135e-12]),
 8.064215961667287e-12)

In [12]:
numParticulas = 70 # Misma población que en AG para que la comparación sea justa.
maxIteraciones = 1000 # Mismas iteraciones que en AG para que la comparación sea justa.
dimension = 10 # Misma dimensión que en AG para que la comparación sea justa.
w = 0.729
c1 = 1.49445
c2 = 1.49445 
funcionObjetivo = Griewank
intervalo = intervaloGriewank

PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([-3.14002225e+00,  4.43844337e+00, -8.42878367e-09,  6.27064096e+00,
        -7.00729236e+00, -1.27585343e-08, -8.78335977e-09, -8.85035719e+00,
        -1.82110578e-08,  9.88515382e+00]),
 0.07376398054413591)

**PSO para análisis estadístico:**

In [13]:
@numba.njit
def PSO_Estadistico(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2):

    # Inicializamos listas donde guardaremos las iteraciones 
    aptitud_promedio = np.zeros(maxIteraciones)
    aptitud_mejor = np.zeros(maxIteraciones)
    
    # Inicializamos nuestra población inicial de partículas y velocidades:
    a, b = intervalo
    particulas = np.random.uniform(a, b, (numParticulas, dimension))
    velocidades = np.zeros((numParticulas, dimension))

    # Inicializamos las mejores posiciones y mejores valores:
    mejores_posiciones = np.copy(particulas) 
    mejores_aptitudes = np.empty(numParticulas)
    for i in range(numParticulas):
        mejores_aptitudes[i] = funcionObjetivo(particulas[i])

    mejor_posicion_enjambre = mejores_posiciones[np.argmin(mejores_aptitudes)]
    mejor_aptitud_enjambre = np.min(mejores_aptitudes)
    mejor_aptitud_factible = 0

    # Contador para verificar cuántas veces la mejor solución global salió del rango
    contador_fuera_de_rango = 0

    # Iteramos hasta maxIteraciones, actualizando la velocidad y la posición de cada partícula en cada iteración:
    for iteracion in range(maxIteraciones):
        # Actualizamos las velocidades:
        r1 = np.random.uniform(0, 1, (numParticulas, dimension))
        r2 = np.random.uniform(0, 1, (numParticulas, dimension))
        for i in range(numParticulas):
            for d in range(dimension):
                velocidades[i, d] = (w * velocidades[i, d] + c1 * r1[i, d] * (mejores_posiciones[i, d] - particulas[i, d]) + c2 * r2[i, d] * (mejor_posicion_enjambre[d] - particulas[i, d]))

        # Actualizamos las posiciones:
        for i in range(numParticulas):
            for d in range(dimension):
                particulas[i, d] += velocidades[i, d]

        # Evaluamos la aptitud de cada partícula:
        evaluaciones_de_aptitud = np.empty(numParticulas)
        for i in range(numParticulas):
            evaluaciones_de_aptitud[i] = funcionObjetivo(particulas[i])

        # Actualizamos las posiciones y las aptitudes:
        for i in range(numParticulas):
            if evaluaciones_de_aptitud[i] < mejores_aptitudes[i]:
                mejores_aptitudes[i] = evaluaciones_de_aptitud[i]
                mejores_posiciones[i] = particulas[i]

        # Actualizamos la mejor posición global si es que encontramos una mejor:
        mejor_aptitud_actual = np.min(evaluaciones_de_aptitud)
        if mejor_aptitud_actual < mejor_aptitud_enjambre:
            mejor_aptitud_enjambre = mejor_aptitud_actual
            mejor_posicion_enjambre = particulas[np.argmin(evaluaciones_de_aptitud)]

        # Verificamos si la mejor posición global está dentro del rango
        fuera_del_rango = False
        for d in range(dimension):
            if mejor_posicion_enjambre[d] < a or mejor_posicion_enjambre[d] > b:
                fuera_del_rango = True
                break

        # Si está fuera del rango, incrementamos el contador
        if fuera_del_rango:
            contador_fuera_de_rango += 1
        # Si sí está en el rango:
        else:
            mejor_aptitud_factible = mejor_aptitud_enjambre

        # Guardamos la mejor evaluación de la generación y la evaluación promedio:
        aptitud_mejor[iteracion] = mejor_aptitud_enjambre
        aptitud_promedio[iteracion] = np.mean(evaluaciones_de_aptitud)

    # Verificamos si en la última iteración la mejor solución global está dentro del intervalo
    ultima_iteracion_en_rango = True
    for d in range(dimension):
        if mejor_posicion_enjambre[d] < a or mejor_posicion_enjambre[d] > b:
            ultima_iteracion_en_rango = False
            break

    if ultima_iteracion_en_rango == True: # Si la última iteración sí se quedó en el espacio de búsqueda:
        # Retornamos la lista de las mejores soluciones encontradas a cada generación, la lista de los promedios generacionales, el contador de las veces que la mejor posición estuvo fuera del rango, y si la última iteración estuvo en rango y la última solución factible encontrada (que va a ser la misma que la obtenida en la iteración final).
        return aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, mejor_aptitud_factible
    else: # En caso de que la última iteracion se haya salido del espacio de búsqueda:
        # Retornamos la lista de las mejores soluciones encontradas a cada generación, la lista de los promedios generacionales, el contador de las veces que la mejor posición estuvo fuera del rango, si la última iteración estuvo en rango y la última solución factible encontrada.
        return aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, mejor_aptitud_factible


In [14]:
PSO_Estadistico(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([66.42605519, 38.07479319, 32.52528901, 24.83363209, 24.83363209,
        24.83363209, 17.8951097 ,  5.85940971,  5.05858089,  5.05858089,
         5.05858089,  5.05858089,  5.05858089,  5.05858089,  5.05858089,
         5.05858089,  5.05858089,  5.05858089,  4.71510914,  3.4803526 ,
         3.25887952,  2.3440466 ,  1.78949451,  1.57646501,  1.57646501,
         1.57646501,  1.57646501,  1.57646501,  1.57646501,  1.57646501,
         1.57646501,  1.57646501,  1.57646501,  1.57646501,  1.57646501,
         1.57646501,  1.47868593,  1.47868593,  1.47868593,  1.26814819,
         1.26801683,  1.26801683,  1.17836806,  1.06722628,  1.06722628,
         1.06722628,  1.06722628,  1.06722628,  1.06722628,  0.88742105,
         0.88742105,  0.88742105,  0.88742105,  0.88742105,  0.88742105,
         0.88742105,  0.88742105,  0.88742105,  0.88742105,  0.88742105,
         0.88742105,  0.88742105,  0.88742105,  0.88742105,  0.88742105,
         0.88742105,  0.88742105,  0.82078874,  0.8

**Análisis estadístico:**

In [108]:
import time
import numpy as np

# Función principal para ejecutar el algoritmo 30 veces y guardar los resultados
def ejecutar_algoritmo_y_guardar_resultados(ejecuciones=30):
    numParticulas = 70  # Tamaño de la población
    maxIteraciones = 1000  # Número de iteraciones
    dimension = 10  # Dimensionalidad del problema
    w = 0.729
    c1 = 1.49445
    c2 = 1.49445 
    funcionObjetivo = Rosenbrock
    intervalo = intervaloRosenbrock

    # Lista para contar cuántas veces la mejor solución se salió del rango
    mejor_adentro_que_afuera = np.zeros(ejecuciones)

    for j in range(ejecuciones):
        # Generar semilla única para cada ejecución
        seed = int(time.time()) % (2**32)
        seed += j
        np.random.seed(seed)

        # Ejecutar el algoritmo PSO
        aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, _ = PSO_Estadistico(
            funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2
        )

        # Convertir `contador_fuera_de_rango` y `ultima_iteracion_en_rango` en listas para coincidir en longitud con los otros resultados
        contador_fuera_de_rango_arr = np.full_like(aptitud_mejor, contador_fuera_de_rango)
        ultima_iteracion_en_rango_arr = np.full_like(aptitud_mejor, ultima_iteracion_en_rango)

        # Guardar resultados en un archivo CSV para cada ejecución
        nombre_archivo = f"Rosenbrock_PSO_{j+1}_semilla_{seed}.csv"
        header = "MejorEval,PromedioEval,ContadorFueraDeRango,UltimaEnRango"

        # Apilar todos los resultados en columnas
        datos = np.column_stack((aptitud_mejor, aptitud_promedio, contador_fuera_de_rango_arr, ultima_iteracion_en_rango_arr))

        # Guardar los datos en un archivo CSV
        np.savetxt(nombre_archivo, datos, delimiter=',', header=header, comments='', fmt='%.6f,%.6f,%d,%s')

# Ejecutar el algoritmo y guardar los resultados
ejecutar_algoritmo_y_guardar_resultados()


In [ ]:
import time
import numpy as np

# Función principal para ejecutar el algoritmo 30 veces y guardar los resultados
def ejecutar_algoritmo_y_guardar_resultados(ejecuciones=30):
    numParticulas = 70  # Tamaño de la población
    maxIteraciones = 1000  # Número de iteraciones
    dimension = 10  # Dimensionalidad del problema
    w = 0.729
    c1 = 1.49445
    c2 = 1.49445 
    funcionObjetivo = Ackley
    intervalo = intervaloAckley

    # Lista para contar cuántas veces la mejor solución se salió del rango
    mejor_adentro_que_afuera = np.zeros(ejecuciones)

    for j in range(ejecuciones):
        # Generar semilla única para cada ejecución
        seed = int(time.time()) % (2**32)
        seed += j
        np.random.seed(seed)

        # Ejecutar el algoritmo PSO
        aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, _ = PSO_Estadistico(
            funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2
        )

        # Convertir `contador_fuera_de_rango` y `ultima_iteracion_en_rango` en listas para coincidir en longitud con los otros resultados
        contador_fuera_de_rango_arr = np.full_like(aptitud_mejor, contador_fuera_de_rango)
        ultima_iteracion_en_rango_arr = np.full_like(aptitud_mejor, ultima_iteracion_en_rango)

        # Guardar resultados en un archivo CSV para cada ejecución
        nombre_archivo = f"Ackley_PSO_{j+1}_semilla_{seed}.csv"
        header = "MejorEval,PromedioEval,ContadorFueraDeRango,UltimaEnRango"

        # Apilar todos los resultados en columnas
        datos = np.column_stack((aptitud_mejor, aptitud_promedio, contador_fuera_de_rango_arr, ultima_iteracion_en_rango_arr))

        # Guardar los datos en un archivo CSV
        np.savetxt(nombre_archivo, datos, delimiter=',', header=header, comments='', fmt='%.6f,%.6f,%d,%s')

# Ejecutar el algoritmo y guardar los resultados
ejecutar_algoritmo_y_guardar_resultados()


In [ ]:
import time
import numpy as np

# Función principal para ejecutar el algoritmo 30 veces y guardar los resultados
def ejecutar_algoritmo_y_guardar_resultados(ejecuciones=30):
    numParticulas = 70  # Tamaño de la población
    maxIteraciones = 1000  # Número de iteraciones
    dimension = 10  # Dimensionalidad del problema
    w = 0.729
    c1 = 1.49445
    c2 = 1.49445 
    funcionObjetivo = Griewank
    intervalo = intervaloGriewank

    # Lista para contar cuántas veces la mejor solución se salió del rango
    mejor_adentro_que_afuera = np.zeros(ejecuciones)

    for j in range(ejecuciones):
        # Generar semilla única para cada ejecución
        seed = int(time.time()) % (2**32)
        seed += j
        np.random.seed(seed)

        # Ejecutar el algoritmo PSO
        aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, _ = PSO_Estadistico(
            funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2
        )

        # Convertir `contador_fuera_de_rango` y `ultima_iteracion_en_rango` en listas para coincidir en longitud con los otros resultados
        contador_fuera_de_rango_arr = np.full_like(aptitud_mejor, contador_fuera_de_rango)
        ultima_iteracion_en_rango_arr = np.full_like(aptitud_mejor, ultima_iteracion_en_rango)

        # Guardar resultados en un archivo CSV para cada ejecución
        nombre_archivo = f"Griewank_PSO_{j+1}_semilla_{seed}.csv"
        header = "MejorEval,PromedioEval,ContadorFueraDeRango,UltimaEnRango"

        # Apilar todos los resultados en columnas
        datos = np.column_stack((aptitud_mejor, aptitud_promedio, contador_fuera_de_rango_arr, ultima_iteracion_en_rango_arr))

        # Guardar los datos en un archivo CSV
        np.savetxt(nombre_archivo, datos, delimiter=',', header=header, comments='', fmt='%.6f,%.6f,%d,%s')

# Ejecutar el algoritmo y guardar los resultados
ejecutar_algoritmo_y_guardar_resultados()


In [ ]:
import time
import numpy as np

# Función principal para ejecutar el algoritmo 30 veces y guardar los resultados
def ejecutar_algoritmo_y_guardar_resultados(ejecuciones=30):
    numParticulas = 70  # Tamaño de la población
    maxIteraciones = 1000  # Número de iteraciones
    dimension = 10  # Dimensionalidad del problema
    w = 0.729
    c1 = 1.49445
    c2 = 1.49445 
    funcionObjetivo = Rastrigin
    intervalo = intervaloRastrigin

    # Lista para contar cuántas veces la mejor solución se salió del rango
    mejor_adentro_que_afuera = np.zeros(ejecuciones)

    for j in range(ejecuciones):
        # Generar semilla única para cada ejecución
        seed = int(time.time()) % (2**32)
        seed += j
        np.random.seed(seed)

        # Ejecutar el algoritmo PSO
        aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, _ = PSO_Estadistico(
            funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2
        )

        # Convertir `contador_fuera_de_rango` y `ultima_iteracion_en_rango` en listas para coincidir en longitud con los otros resultados
        contador_fuera_de_rango_arr = np.full_like(aptitud_mejor, contador_fuera_de_rango)
        ultima_iteracion_en_rango_arr = np.full_like(aptitud_mejor, ultima_iteracion_en_rango)

        # Guardar resultados en un archivo CSV para cada ejecución
        nombre_archivo = f"Rastrigin_PSO_{j+1}_semilla_{seed}.csv"
        header = "MejorEval,PromedioEval,ContadorFueraDeRango,UltimaEnRango"

        # Apilar todos los resultados en columnas
        datos = np.column_stack((aptitud_mejor, aptitud_promedio, contador_fuera_de_rango_arr, ultima_iteracion_en_rango_arr))

        # Guardar los datos en un archivo CSV
        np.savetxt(nombre_archivo, datos, delimiter=',', header=header, comments='', fmt='%.6f,%.6f,%d,%s')

# Ejecutar el algoritmo y guardar los resultados
ejecutar_algoritmo_y_guardar_resultados()


In [ ]:
import time
import numpy as np

# Función principal para ejecutar el algoritmo 30 veces y guardar los resultados
def ejecutar_algoritmo_y_guardar_resultados(ejecuciones=30):
    numParticulas = 70  # Tamaño de la población
    maxIteraciones = 1000  # Número de iteraciones
    dimension = 10  # Dimensionalidad del problema
    w = 0.729
    c1 = 1.49445
    c2 = 1.49445 
    funcionObjetivo = Sphere
    intervalo = intervaloSphere

    # Lista para contar cuántas veces la mejor solución se salió del rango
    mejor_adentro_que_afuera = np.zeros(ejecuciones)

    for j in range(ejecuciones):
        # Generar semilla única para cada ejecución
        seed = int(time.time()) % (2**32)
        seed += j
        np.random.seed(seed)

        # Ejecutar el algoritmo PSO
        aptitud_mejor, aptitud_promedio, contador_fuera_de_rango, ultima_iteracion_en_rango, _ = PSO_Estadistico(
            funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2
        )

        # Convertir `contador_fuera_de_rango` y `ultima_iteracion_en_rango` en listas para coincidir en longitud con los otros resultados
        contador_fuera_de_rango_arr = np.full_like(aptitud_mejor, contador_fuera_de_rango)
        ultima_iteracion_en_rango_arr = np.full_like(aptitud_mejor, ultima_iteracion_en_rango)

        # Guardar resultados en un archivo CSV para cada ejecución
        nombre_archivo = f"Sphere_PSO_{j+1}_semilla_{seed}.csv"
        header = "MejorEval,PromedioEval,ContadorFueraDeRango,UltimaEnRango"

        # Apilar todos los resultados en columnas
        datos = np.column_stack((aptitud_mejor, aptitud_promedio, contador_fuera_de_rango_arr, ultima_iteracion_en_rango_arr))

        # Guardar los datos en un archivo CSV
        np.savetxt(nombre_archivo, datos, delimiter=',', header=header, comments='', fmt='%.6f,%.6f,%d,%s')

# Ejecutar el algoritmo y guardar los resultados
ejecutar_algoritmo_y_guardar_resultados()


**Tabla PSO:**

In [6]:
import os
import pandas as pd

# Directorio base donde se encuentran las carpetas de funciones
base_dir = r"C:\Users\Chinitok666\Documents\GitHub\C-mputo-Evolutivo-primeros-intentos\PSO"

# Lista de nombres de las carpetas para cada función
funciones = ['Ejecuciones_Ackley', 'Ejecuciones_Griewank', 'Ejecuciones_Rastrigin', 'Ejecuciones_Rosenbrock', 'Ejecuciones_Sphere']

# Lista para almacenar los resultados de cada función
resultados = []

# Recorrer cada carpeta de función
for funcion in funciones:
    carpeta_funcion = os.path.join(base_dir, funcion)
    valores_mejor_eval = []
    dentro_rango = 0  # Contador de ejecuciones dentro del rango

    # Verificar si la carpeta existe
    if not os.path.exists(carpeta_funcion):
        print(f"Carpeta no encontrada: {carpeta_funcion}")
        continue

    print(f"\nProcesando función: {funcion}")
    
    # Procesar cada archivo CSV en la carpeta de la función
    archivos_encontrados = 0
    for archivo in os.listdir(carpeta_funcion):
        if archivo.endswith('.csv'):
            archivos_encontrados += 1
            
            # Leer el archivo CSV
            archivo_path = os.path.join(carpeta_funcion, archivo)
            df = pd.read_csv(archivo_path)
            
            # Verificar que el archivo no esté vacío
            if df.empty:
                print(f"Archivo vacío: {archivo_path}")
                continue
            
            # Extraer el valor de la última fila de la primera columna (MejorEval)
            try:
                mejor_eval = df.iloc[-1, 0]
                valores_mejor_eval.append(mejor_eval)
                print(f"  {archivo}: MejorEval final = {mejor_eval}")
            except IndexError:
                print(f"  Error al leer MejorEval en {archivo_path}")
                continue
            
            # Comprobar si la última fila de 'UltimaEnRango' es 1 (en rango)
            try:
                ultima_en_rango = df.iloc[-1, -1]  # Última columna
                if ultima_en_rango == 1:
                    dentro_rango += 1
            except IndexError:
                print(f"  Error al leer UltimaEnRango en {archivo_path}")

    if archivos_encontrados == 0:
        print(f"No se encontraron archivos CSV en {carpeta_funcion}")
        continue

    # Calcular estadísticas solo si hay datos
    if valores_mejor_eval:
        mejor_valor = min(valores_mejor_eval)
        promedio = sum(valores_mejor_eval) / len(valores_mejor_eval)
        peor_valor = max(valores_mejor_eval)
        mediana = pd.Series(valores_mejor_eval).median()
        desviacion_std = pd.Series(valores_mejor_eval).std()
        porcentaje_dentro_rango = (dentro_rango / len(valores_mejor_eval)) * 100

        # Almacenar resultados
        resultados.append({
            'Funcion': funcion,
            'Mejor Valor': mejor_valor,
            'Promedio': promedio,
            'Peor Valor': peor_valor,
            'Mediana': mediana,
            'Desviación Estándar': desviacion_std,
            'Porcentaje en Rango': porcentaje_dentro_rango
        })
    else:
        print(f"No se encontraron datos válidos para la función: {funcion}")

# Crear un DataFrame y guardarlo como CSV
df_resultados = pd.DataFrame(resultados)
if not df_resultados.empty:
    df_resultados.to_csv('resultados_estadisticos.csv', index=False)
    print("\nArchivo 'resultados_estadisticos.csv' generado con éxito.")
else:
    print("\nNo se generaron resultados estadísticos, la tabla está vacía.")



Procesando función: Ejecuciones_Ackley
  Ackley_PSO_10_semilla_1730873128.csv: MejorEval final = 0.0
  Ackley_PSO_11_semilla_1730873129.csv: MejorEval final = 0.0
  Ackley_PSO_12_semilla_1730873130.csv: MejorEval final = 0.0
  Ackley_PSO_13_semilla_1730873131.csv: MejorEval final = 0.0
  Ackley_PSO_14_semilla_1730873132.csv: MejorEval final = 0.0
  Ackley_PSO_15_semilla_1730873133.csv: MejorEval final = 0.0
  Ackley_PSO_16_semilla_1730873134.csv: MejorEval final = 0.0
  Ackley_PSO_17_semilla_1730873135.csv: MejorEval final = 0.0
  Ackley_PSO_18_semilla_1730873136.csv: MejorEval final = 0.0
  Ackley_PSO_19_semilla_1730873137.csv: MejorEval final = 0.0
  Ackley_PSO_1_semilla_1730873118.csv: MejorEval final = 0.0
  Ackley_PSO_20_semilla_1730873138.csv: MejorEval final = 0.0
  Ackley_PSO_21_semilla_1730873139.csv: MejorEval final = 0.0
  Ackley_PSO_22_semilla_1730873140.csv: MejorEval final = 0.0
  Ackley_PSO_23_semilla_1730873141.csv: MejorEval final = 0.0
  Ackley_PSO_24_semilla_1730873

**PSO que no se sale del espacio de búsqueda:**

In [15]:
@numba.njit
def PSO_Estadistico(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2):

    # Inicializamos listas donde guardaremos las iteraciones 
    aptitud_promedio = np.zeros(maxIteraciones)
    aptitud_mejor = np.zeros(maxIteraciones)
    
    # Inicializamos nuestra población inicial de partículas y velocidades:
    a, b = intervalo
    particulas = np.random.uniform(a, b, (numParticulas, dimension))
    velocidades = np.zeros((numParticulas, dimension))

    # Inicializamos las mejores posiciones y mejores valores:
    mejores_posiciones = np.copy(particulas)
    mejores_aptitudes = np.empty(numParticulas)
    for i in range(numParticulas):
        mejores_aptitudes[i] = funcionObjetivo(particulas[i])

    mejor_posicion_enjambre = mejores_posiciones[np.argmin(mejores_aptitudes)]
    mejor_aptitud_enjambre = np.min(mejores_aptitudes)

    # Iteramos hasta maxIteraciones, actualizando la velocidad y la posición de cada partícula en cada iteración:
    for iteracion in range(maxIteraciones):
        # Actualizamos las velocidades:
        r1 = np.random.uniform(0, 1, (numParticulas, dimension))
        r2 = np.random.uniform(0, 1, (numParticulas, dimension))
        for i in range(numParticulas):
            for d in range(dimension):
                velocidades[i, d] = (w * velocidades[i, d] + 
                                     c1 * r1[i, d] * (mejores_posiciones[i, d] - particulas[i, d]) +
                                     c2 * r2[i, d] * (mejor_posicion_enjambre[d] - particulas[i, d]))

        # Actualizamos las posiciones:
        for i in range(numParticulas):
            for d in range(dimension):
                particulas[i, d] += velocidades[i, d]

        # Aplicamos la restricción para mantener las partículas dentro del intervalo [a, b]
        particulas = np.clip(particulas, a, b)

        # Evaluamos la aptitud de cada partícula:
        evaluaciones_de_aptitud = np.empty(numParticulas)
        for i in range(numParticulas):
            evaluaciones_de_aptitud[i] = funcionObjetivo(particulas[i])

        # Actualizamos las posiciones y las aptitudes:
        for i in range(numParticulas):
            if evaluaciones_de_aptitud[i] < mejores_aptitudes[i]:
                mejores_aptitudes[i] = evaluaciones_de_aptitud[i]
                mejores_posiciones[i] = particulas[i]

        # Actualizamos la mejor posición global si es que encontramos una mejor:
        mejor_aptitud_actual = np.min(evaluaciones_de_aptitud)
        if mejor_aptitud_actual < mejor_aptitud_enjambre:
            mejor_aptitud_enjambre = mejor_aptitud_actual
            mejor_posicion_enjambre = particulas[np.argmin(evaluaciones_de_aptitud)]

        # Guardamos la mejor aptitud y el promedio de aptitud de la generación actual:
        aptitud_mejor[iteracion] = mejor_aptitud_enjambre
        aptitud_promedio[iteracion] = np.mean(evaluaciones_de_aptitud)

    # Retornamos la lista de las mejores soluciones encontradas en cada generación y la lista de los promedios generacionales:
    return aptitud_mejor, aptitud_promedio


In [125]:
numParticulas = 70 # Misma población que en AG para que la comparación sea justa.
maxIteraciones = 1000 # Mismas iteraciones que en AG para que la comparación sea justa.
dimension = 10 # Misma dimensión que en AG para que la comparación sea justa.
w = 0.729
c1 = 1.49445
c2 = 1.49445 
funcionObjetivo = Rastrigin
intervalo = intervaloRastrigin

PSO(funcionObjetivo, intervalo, dimension, numParticulas, maxIteraciones, w, c1, c2)


(array([-1.38439015e-09,  9.94958636e-01,  1.98991223e+00,  9.94958639e-01,
        -9.94958637e-01,  9.94958639e-01, -5.35715289e-10,  5.80254525e-10,
        -1.10228578e-09, -8.30956021e-10]),
 7.959667418927239)